## Data Prepare

### data ready

In [12]:
from DataProcess.data_extract import load_all_data, extract_main_tasks, extract_nasatlx_data, extract_dif_conf_data, extract_post_study_responses,extract_quiz_data, normalize_typing_time,split_by_empty_answer,combine_modifying_results
from DataProcess.reading_analysis import annotate_reading, aggregate_reading
from DataProcess.visualization import compare_tasks_across_formats
from DataProcess.parser import run_all_validation
from DataProcess.parse_to_json import CleanJSONParser, CleanXMLParser, CleanYTParser, parse_all_answers
from DataProcess.semantic_evaluate import DeepDiffMetric, TreeEditDistanceMetric, TruthDiffMetric
import json
import pandas as pd

# data folder: config/tabular
folder ='/Users/shiyi.he/Desktop/DSF_Analysis/config'
# folder ='/Users/shiyi.he/Desktop/DSF_Analysis/tabular'

correct_answers_config = {
    'reading-task-config-1': '7.9.0',
    'reading-task-config-2': ">=12",
    'reading-task-config-3': 6,
    'reading-task-config-4': 'keywords',
    'reading-task-config-5': 'd3-delaunay'
}

correct_answers_tabular = {
    'reading-task-tabular-1': 87,
    'reading-task-tabular-2': 'quiz2',
    'reading-task-tabular-3': 4,
    'reading-task-tabular-4': 'Bob',
    'reading-task-tabular-5': ['[Bob, Alice]', '[Eve]']
}

## read all data from the folder
data = load_all_data(folder, False)

## extract main tasks data
df_reading, df_writing, df_writing_nl, df_modifying = extract_main_tasks(data)

## extract nasatlx, survey, quizdata
df_nasa=extract_nasatlx_data(data)
df_dif_conf = extract_dif_conf_data(data)
df_post =  extract_post_study_responses(data)
quiz_results=extract_quiz_data(data)


### prepare reading result

In [1]:
reading_useful_data=df_reading[['participantId','task','format','answer','duration_sec','help_count','search_count','copy_count','paste_count']]

### result data that we need for reading
reading_result =annotate_reading(
    reading_useful_data,
    correct_answers_config)

reading_aggregated_result = aggregate_reading(reading_result)

reading_result.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/config_result/reading_result.csv')
reading_aggregated_result.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/config_result/reading_aggregated_result.csv')

NameError: name 'df_reading' is not defined

In [ ]:
## tabular
reading_useful_data=df_reading[['participantId','task','format','answer','duration_sec','help_count','search_count','copy_count','paste_count']]

### result data that we need for reading
reading_result =annotate_reading(
    reading_useful_data,
    correct_answers_tabular)

reading_aggregated_result = aggregate_reading(reading_result)

reading_result.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/tabular_result/reading_result.csv')
reading_aggregated_result.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/tabular_result/reading_aggregated_result.csv')
reading_aggregated_result

### prepare writing result

In [ ]:
### normalize the typing time
df_writing_norm = normalize_typing_time(df_writing_nl, df_writing)

### run the validation
df_writing_parse_result = run_all_validation(df_writing)


### 把所有的用户答案转换成json格式
parser_json = CleanJSONParser()
parser_xml  = CleanXMLParser()
parser_ytt  = CleanYTParser()
df_writing_parsed = parse_all_answers(df_writing, parser_json, parser_xml, parser_ytt)
df_writing_parsed

## tree similarity analysis
with open('ground_truth_writing_config.json', 'r', encoding='utf-8') as f:  # update the path!!!!!更改路径tabular/config
    ground_truth_writing_config = json.load(f)

w_ts_result = TreeEditDistanceMetric.run_semantic_pipeline(df_writing_parsed, ground_truth_writing_config)
w_ts_result_writing=w_ts_result[['participantId','format','task','tree_similarity']]

### run DeepDiff的结果
import json
gt_path='ground_truth_writing_config.json'     # update the path!!!!!更改路径tabular/config
with open(gt_path) as f:
    G = json.load(f)

w_DD_result=TruthDiffMetric.evaluate_answers_against_truth(G, 
    df_writing_parsed,
    parsed_col="parsed_answer",
    id_col="participantId",
    fmt_col="format")
w_DD_result_writing=w_DD_result[['participantId','format','precision','recall','f1']]
w_DD_result_writing['task']='writing-task-config'  # update the path!!!!!更改路径tabular/config

### merge the result
semantic_merged = w_DD_result_writing.merge(
    w_ts_result_writing[['participantId','format','task','tree_similarity']],
    on=['participantId','format','task'],
    how='left'
)

parse_semantic_merged = semantic_merged.merge(df_writing_parse_result,on=['participantId','format','task'],how='left')

full_merged=parse_semantic_merged.merge(df_writing_norm,on=['participantId','format','task'],how='left')

full_merged.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/config_result/writing_result.csv')   # update the path!!!!!更改路径tabular/config

In [ ]:
### normalize the typing time
df_writing_norm = normalize_typing_time(df_writing_nl, df_writing)

### run the validation
df_writing_parse_result = run_all_validation(df_writing)


### 把所有的用户答案转换成json格式
parser_json = CleanJSONParser()
parser_xml  = CleanXMLParser()
parser_ytt  = CleanYTParser()
df_writing_parsed = parse_all_answers(df_writing, parser_json, parser_xml, parser_ytt)

## tree similarity analysis
with open('ground_truth_writing_tabular.json', 'r', encoding='utf-8') as f:  # update the path!!!!!更改路径tabular/config
    ground_truth_writing_tabular = json.load(f)

w_ts_result = TreeEditDistanceMetric.run_semantic_pipeline(df_writing_parsed, ground_truth_writing_tabular)
w_ts_result_writing=w_ts_result[['participantId','format','task','tree_similarity']]

### run DeepDiff的结果
import json
gt_path='ground_truth_writing_tabular.json'     # update the path!!!!!更改路径tabular/config
with open(gt_path) as f:
    G = json.load(f)

w_DD_result=TruthDiffMetric.evaluate_answers_against_truth(G, 
    df_writing_parsed,
    parsed_col="parsed_answer",
    id_col="participantId",
    fmt_col="format")
w_DD_result_writing=w_DD_result[['participantId','format','precision','recall','f1']]
w_DD_result_writing['task']='writing-task-tabular'  # update the path!!!!!更改路径tabular/config

### merge the result
semantic_merged = w_DD_result_writing.merge(
    w_ts_result_writing[['participantId','format','task','tree_similarity']],
    on=['participantId','format','task'],
    how='left'
)

parse_semantic_merged = semantic_merged.merge(df_writing_parse_result,on=['participantId','format','task'],how='left')

full_merged=parse_semantic_merged.merge(df_writing_norm,on=['participantId','format','task'],how='left')

full_merged.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/tabular_result/writing_result.csv')   # update the path!!!!!更改路径tabular/config
full_merged

### prepare modifying data

#### config

In [ ]:
## 处理空值
modifying_empty, modifying_nonempty = split_by_empty_answer(df_modifying)
modifying_empty['normalized_time']=modifying_empty['duration_sec']

df_modifying_norm=modifying_nonempty[['participantId','format','task','duration_sec','help_count','search_count','copy_count','paste_count']]
df_modifying_norm['normalized_time']=df_modifying_norm['duration_sec']

### run the validation
df_modifying_parse_result = run_all_validation(modifying_nonempty)


### 把所有的用户答案转换成json格式
parser_json = CleanJSONParser()
parser_xml  = CleanXMLParser()
parser_ytt  = CleanYTParser()
df_modifying_parsed = parse_all_answers(modifying_nonempty, parser_json, parser_xml, parser_ytt)

df_modifying_parsed_1 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-config-1']
df_modifying_parsed_2 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-config-2']
df_modifying_parsed_3 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-config-3']
df_modifying_parsed_4 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-config-4']


## tree similarity analysis
### task 1

with open('ground_truth_modifying_config_1.json', 'r', encoding='utf-8') as f:  # update the path!!!!!更改路径tabular/config
    ground_truth_modifying_config_1 = json.load(f)

df_modifying_parsed_1 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-config-1']
m_1_ts_result = TreeEditDistanceMetric.run_semantic_pipeline(df_modifying_parsed_1, ground_truth_modifying_config_1)
m_1_ts_result_modifying=m_1_ts_result[['participantId','format','task','tree_similarity']]

with open('ground_truth_modifying_config_2.json', 'r', encoding='utf-8') as f:  # update the path!!!!!更改路径tabular/config
    ground_truth_modifying_config_2 = json.load(f)

df_modifying_parsed_2 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-config-2']
m_2_ts_result = TreeEditDistanceMetric.run_semantic_pipeline(df_modifying_parsed_2, ground_truth_modifying_config_2)
m_2_ts_result_modifying=m_2_ts_result[['participantId','format','task','tree_similarity']]

with open('ground_truth_modifying_config_3.json', 'r', encoding='utf-8') as f:  # update the path!!!!!更改路径tabular/config
    ground_truth_modifying_config_3 = json.load(f)

df_modifying_parsed_3 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-config-3']
m_3_ts_result = TreeEditDistanceMetric.run_semantic_pipeline(df_modifying_parsed_3, ground_truth_modifying_config_3)
m_3_ts_result_modifying=m_3_ts_result[['participantId','format','task','tree_similarity']]

with open('ground_truth_modifying_config_4.json', 'r', encoding='utf-8') as f:  # update the path!!!!!更改路径tabular/config
    ground_truth_modifying_config_4 = json.load(f)

df_modifying_parsed_4 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-config-4']
m_4_ts_result = TreeEditDistanceMetric.run_semantic_pipeline(df_modifying_parsed_4, ground_truth_modifying_config_4)
m_4_ts_result_modifying=m_4_ts_result[['participantId','format','task','tree_similarity']]

m_ts_result_modifying=pd.concat([m_1_ts_result_modifying,m_2_ts_result_modifying,m_3_ts_result_modifying,m_4_ts_result_modifying])

### run DeepDiff的结果


prompt_path='prompt_modifying_config.json' # update the path!!!!!更改路径tabular/config
with open(prompt_path) as f:
    P = json.load(f)

gt_path_1='ground_truth_modifying_config_1.json' # update the path!!!!!更改路径tabular/config
with open(gt_path_1) as f:
    G_1 = json.load(f)

gt_path_2='ground_truth_modifying_config_2.json' # update the path!!!!!更改路径tabular/config
with open(gt_path_2) as f:
    G_2 = json.load(f)

gt_path_3='ground_truth_modifying_config_3.json' # update the path!!!!!更改路径tabular/config
with open(gt_path_3) as f:
    G_3 = json.load(f)

gt_path_4='ground_truth_modifying_config_4.json' # update the path!!!!!更改路径tabular/config
with open(gt_path_4) as f:
    G_4 = json.load(f)

m_1_DD_result=DeepDiffMetric.evaluate_answers(P, G_1, df_modifying_parsed_1,ignore_keys=["homepage","collaborators","scripts"]) # update the path!!!!!更改路径tabular/config
m_1_DD_result_modifying=m_1_DD_result[['participantId','format','precision','recall','f1']]
m_1_DD_result_modifying['task']='modifying-task-config-1' # update the path!!!!!更改路径tabular/config

m_2_DD_result=DeepDiffMetric.evaluate_answers(P, G_2, df_modifying_parsed_2,ignore_keys=["homepage","collaborators","scripts"]) # update the path!!!!!更改路径tabular/config
m_2_DD_result_modifying=m_2_DD_result[['participantId','format','precision','recall','f1']]
m_2_DD_result_modifying['task']='modifying-task-config-2' # update the path!!!!!更改路径tabular/config

m_3_DD_result=DeepDiffMetric.evaluate_answers(P, G_3, df_modifying_parsed_3,ignore_keys=["homepage","collaborators","scripts"]) # update the path!!!!!更改路径tabular/config
m_3_DD_result_modifying=m_3_DD_result[['participantId','format','precision','recall','f1']]
m_3_DD_result_modifying['task']='modifying-task-config-3' # update the path!!!!!更改路径tabular/config

m_4_DD_result=DeepDiffMetric.evaluate_answers(P, G_4, df_modifying_parsed_4,ignore_keys=["homepage","collaborators","scripts"]) # update the path!!!!!更改路径tabular/config
m_4_DD_result_modifying=m_4_DD_result[['participantId','format','precision','recall','f1']]
m_4_DD_result_modifying['task']='modifying-task-config-4' # update the path!!!!!更改路径tabular/config

m_DD_result_modifying=pd.concat([m_1_DD_result_modifying,m_2_DD_result_modifying,m_3_DD_result_modifying,m_4_DD_result_modifying])

### merge the result
semantic_merged = m_DD_result_modifying.merge(
    m_ts_result_modifying[['participantId','format','task','tree_similarity']],
    on=['participantId','format','task'],
    how='left'
)

parse_semantic_merged = semantic_merged.merge(df_modifying_parse_result,on=['participantId','format','task'],how='left')

full_merged=parse_semantic_merged.merge(df_modifying_norm,on=['participantId','format','task'],how='left')

df_modifying_all = combine_modifying_results(full_merged, modifying_empty)

df_modifying_all.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/config_result/modifying_result.csv')   # update the path!!!!!更改路径tabular/config
df_modifying_all

#### tabular

In [13]:
### normalize the typing time
df_writing_norm = normalize_typing_time(df_writing_nl, df_writing)

### run the validation
df_writing_parse_result = run_all_validation(df_writing)
df_writing_parse_result

,participantId,format,task,strict_parse,loose_parse
0,659960f32faa4ff4b116de92,toml,writing-task-config,False,False
1,5f7f239e9b204e05b6c36538,json5,writing-task-config,False,False
2,676d220cd6e7101ccba18ba5,jsonc,writing-task-config,False,False
3,67b8636acd5ed420cc098b64,toml,writing-task-config,False,False
4,57ca4b09993b08000192297b,xml,writing-task-config,False,True
...,...,...,...,...,...
100,6693d3ae2bd15cf7a058e6a7,json5,writing-task-config,False,False
101,60e2bf0aa568b97e8d525872,toml,writing-task-config,True,True
102,67d24179c5bd87691f1fe036,yaml,writing-task-config,True,True
103,5c4f5967aac8be0001716a65,xml,writing-task-config,False,True


In [14]:
## 处理空值
modifying_empty, modifying_nonempty = split_by_empty_answer(df_modifying)
modifying_empty['normalized_time']=modifying_empty['duration_sec']

df_modifying_norm=modifying_nonempty[['participantId','format','task','duration_sec','help_count','search_count','copy_count','paste_count']]
df_modifying_norm['normalized_time']=df_modifying_norm['duration_sec']

### run the validation
df_modifying_parse_result = run_all_validation(modifying_nonempty)
df_modifying_parse_result

,participantId,format,task,strict_parse,loose_parse
0,659960f32faa4ff4b116de92,toml,modifying-task-config-1,True,True
1,659960f32faa4ff4b116de92,toml,modifying-task-config-2,True,True
2,659960f32faa4ff4b116de92,toml,modifying-task-config-3,True,True
3,659960f32faa4ff4b116de92,toml,modifying-task-config-4,True,True
4,5f7f239e9b204e05b6c36538,json5,modifying-task-config-1,False,False
...,...,...,...,...,...
415,5c4f5967aac8be0001716a65,xml,modifying-task-config-4,True,True
416,6680071d60b3642bcf5dad37,jsonc,modifying-task-config-1,True,True
417,6680071d60b3642bcf5dad37,jsonc,modifying-task-config-2,False,False
418,6680071d60b3642bcf5dad37,jsonc,modifying-task-config-3,True,True


In [15]:
tabular_parse_result=pd.concat([df_writing_parse_result,df_modifying_parse_result])
tabular_parse_result.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/config_parse_result.csv',index=False)

In [18]:
config_json=pd.read_csv('/Users/shiyi.he/Desktop/llm_config/JSON_CONFIG.csv')
config_jsonc=pd.read_csv('/Users/shiyi.he/Desktop/llm_config/JSONC_CONFIG.csv')
config_json5=pd.read_csv('/Users/shiyi.he/Desktop/llm_config/JSON5_CONFIG.csv')
config_hjson=pd.read_csv('/Users/shiyi.he/Desktop/llm_config/HJSON_CONFIG.csv')
config_xml=pd.read_csv('/Users/shiyi.he/Desktop/llm_config/XML_CONFIG.csv')
config_yaml=pd.read_csv('/Users/shiyi.he/Desktop/llm_config/YAML_CONFIG.csv')
config_toml=pd.read_csv('/Users/shiyi.he/Desktop/llm_config/TOML_CONFIG.csv')
config=pd.concat([config_json,config_jsonc,config_json5,config_hjson,config_xml,config_yaml,config_toml])
config

,Format,TaskType,SubTaskID,Output,TimeUsed(s)
0,JSON,Reading,1,7.9.0,0.00
1,JSON,Reading,2,>=12,0.00
2,JSON,Reading,3,6,0.00
3,JSON,Reading,4,"license, homepage, keywords, name, version, de...",0.00
4,JSON,Reading,5,"['d3', 'd3-array', 'd3-axis', 'd3-brush', 'd3-...",0.00
...,...,...,...,...,...
5,TOML,Writing,1,[SKIPPED - Image Unavailable],0.00
6,TOML,Modifying,1,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.01
7,TOML,Modifying,2,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.01
8,TOML,Modifying,3,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.01


In [21]:
modifying_nonempty

,participantId,format,task,answer,duration_sec,help_count,search_count,copy_count,paste_count
0,659960f32faa4ff4b116de92,toml,modifying-task-config-1,"name = ""vega-lite""\nauthor = ""Dominik Moritz, ...",100.503,0,0,1,1
1,659960f32faa4ff4b116de92,toml,modifying-task-config-2,"name = ""vega-lite""\nauthor = ""Dominik Moritz, ...",139.587,0,0,0,0
2,659960f32faa4ff4b116de92,toml,modifying-task-config-3,"name = ""vega-lite""\nauthor = ""Dominik Moritz, ...",67.235,0,0,0,0
3,659960f32faa4ff4b116de92,toml,modifying-task-config-4,"name = ""vega-lite""\nauthor = ""Dominik Moritz, ...",286.226,1,0,3,4
4,5f7f239e9b204e05b6c36538,json5,modifying-task-config-1,"{\n name: 'vega-lite',\n author: 'Domini...",320.421,2,0,0,0
...,...,...,...,...,...,...,...,...,...
415,5c4f5967aac8be0001716a65,xml,modifying-task-config-4,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",499.679,1,1,4,7
416,6680071d60b3642bcf5dad37,jsonc,modifying-task-config-1,"{\n ""name"": ""vega-lite"",\n ""author"": ""Domini...",158.261,0,0,0,0
417,6680071d60b3642bcf5dad37,jsonc,modifying-task-config-2,"{\n ""name"": ""vega-lite"",\n ""author"": ""Domini...",231.195,0,0,0,0
418,6680071d60b3642bcf5dad37,jsonc,modifying-task-config-3,"{\n ""name"": ""vega-lite"",\n ""author"": ""Domini...",365.453,2,0,0,0


In [26]:
config_wm['participantId'] = range(len(config_wm))
config_wm['task']='config'
config_wm.rename(columns={'Format':'format'},inplace=True)
config_wm

/var/folders/z4/3vn4jgtj71n_hdpgm6b1r_ww0000gp/T/ipykernel_92058/4069611980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  config_wm['participantId'] = range(len(config_wm))
/var/folders/z4/3vn4jgtj71n_hdpgm6b1r_ww0000gp/T/ipykernel_92058/4069611980.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  config_wm['task']='config'
/var/folders/z4/3vn4jgtj71n_hdpgm6b1r_ww0000gp/T/ipykernel_92058/4069611980.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

,format,TaskType,SubTaskID,Output,TimeUsed(s),participantId,task
5,JSON,Writing,1,[SKIPPED - Image Unavailable],0.00,0,config
6,JSON,Modifying,1,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00,1,config
7,JSON,Modifying,2,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00,2,config
8,JSON,Modifying,3,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00,3,config
9,JSON,Modifying,4,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00,4,config
5,JSONC,Writing,1,[SKIPPED - Image Unavailable],0.00,5,config
6,JSONC,Modifying,1,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00,6,config
7,JSONC,Modifying,2,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00,7,config
8,JSONC,Modifying,3,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00,8,config
9,JSONC,Modifying,4,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00,9,config


In [27]:
config_wm_valid = run_all_validation(config_wm, code_col = 'Output')
config_wm_valid

,participantId,format,task,strict_parse,loose_parse
5,0,JSON,config,False,False
6,1,JSON,config,False,True
7,2,JSON,config,False,True
8,3,JSON,config,False,True
9,4,JSON,config,False,True
5,5,JSONC,config,False,False
6,6,JSONC,config,False,True
7,7,JSONC,config,False,True
8,8,JSONC,config,False,True
9,9,JSONC,config,False,True


In [22]:
config_wm = config[config['TaskType'].isin(['Writing', 'Modifying'])]
config_wm

,Format,TaskType,SubTaskID,Output,TimeUsed(s)
5,JSON,Writing,1,[SKIPPED - Image Unavailable],0.00
6,JSON,Modifying,1,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00
7,JSON,Modifying,2,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00
8,JSON,Modifying,3,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00
9,JSON,Modifying,4,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00
5,JSONC,Writing,1,[SKIPPED - Image Unavailable],0.00
6,JSONC,Modifying,1,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00
7,JSONC,Modifying,2,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00
8,JSONC,Modifying,3,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00
9,JSONC,Modifying,4,"{'name': 'vega-lite', 'author': 'Dominik Morit...",0.00


In [2]:
## 处理空值
modifying_empty, modifying_nonempty = split_by_empty_answer(df_modifying)
modifying_empty['normalized_time']=modifying_empty['duration_sec']

df_modifying_norm=modifying_nonempty[['participantId','format','task','duration_sec','help_count','search_count','copy_count','paste_count']]
df_modifying_norm['normalized_time']=df_modifying_norm['duration_sec']

### run the validation
df_modifying_parse_result = run_all_validation(modifying_nonempty)


### 把所有的用户答案转换成json格式
parser_json = CleanJSONParser()
parser_xml  = CleanXMLParser()
parser_ytt  = CleanYTParser()
df_modifying_parsed = parse_all_answers(modifying_nonempty, parser_json, parser_xml, parser_ytt)

df_modifying_parsed_1 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-tabular-1']
df_modifying_parsed_2 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-tabular-2']
df_modifying_parsed_3 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-tabular-3']
df_modifying_parsed_4 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-tabular-4']


## tree similarity analysis
### task 1

with open('ground_truth_modifying_tabular_1.json', 'r', encoding='utf-8') as f:  # update the path!!!!!更改路径tabular/config
    ground_truth_modifying_tabular_1 = json.load(f)

df_modifying_parsed_1 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-tabular-1']
m_1_ts_result = TreeEditDistanceMetric.run_semantic_pipeline(df_modifying_parsed_1, ground_truth_modifying_tabular_1)
m_1_ts_result_modifying=m_1_ts_result[['participantId','format','task','tree_similarity']]

with open('ground_truth_modifying_tabular_2.json', 'r', encoding='utf-8') as f:  # update the path!!!!!更改路径tabular/config
    ground_truth_modifying_tabular_2 = json.load(f)

df_modifying_parsed_2 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-tabular-2']
m_2_ts_result = TreeEditDistanceMetric.run_semantic_pipeline(df_modifying_parsed_2, ground_truth_modifying_tabular_2)
m_2_ts_result_modifying=m_2_ts_result[['participantId','format','task','tree_similarity']]

with open('ground_truth_modifying_tabular_3.json', 'r', encoding='utf-8') as f:  # update the path!!!!!更改路径tabular/config
    ground_truth_modifying_tabular_3 = json.load(f)

df_modifying_parsed_3 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-tabular-3']
m_3_ts_result = TreeEditDistanceMetric.run_semantic_pipeline(df_modifying_parsed_3, ground_truth_modifying_tabular_3)
m_3_ts_result_modifying=m_3_ts_result[['participantId','format','task','tree_similarity']]

with open('ground_truth_modifying_tabular_4.json', 'r', encoding='utf-8') as f:  # update the path!!!!!更改路径tabular/config
    ground_truth_modifying_tabular_4 = json.load(f)

df_modifying_parsed_4 = df_modifying_parsed[df_modifying_parsed['task']=='modifying-task-tabular-4']
m_4_ts_result = TreeEditDistanceMetric.run_semantic_pipeline(df_modifying_parsed_4, ground_truth_modifying_tabular_4)
m_4_ts_result_modifying=m_4_ts_result[['participantId','format','task','tree_similarity']]

m_ts_result_modifying=pd.concat([m_1_ts_result_modifying,m_2_ts_result_modifying,m_3_ts_result_modifying,m_4_ts_result_modifying])

### run DeepDiff的结果


prompt_path='prompt_modifying_tabular.json' # update the path!!!!!更改路径tabular/config
with open(prompt_path) as f:
    P = json.load(f)

gt_path_1='ground_truth_modifying_tabular_1.json' # update the path!!!!!更改路径tabular/config
with open(gt_path_1) as f:
    G_1 = json.load(f)

gt_path_2='ground_truth_modifying_tabular_2.json' # update the path!!!!!更改路径tabular/config
with open(gt_path_2) as f:
    G_2 = json.load(f)

gt_path_3='ground_truth_modifying_tabular_3.json' # update the path!!!!!更改路径tabular/config
with open(gt_path_3) as f:
    G_3 = json.load(f)

gt_path_4='ground_truth_modifying_tabular_4.json' # update the path!!!!!更改路径tabular/config
with open(gt_path_4) as f:
    G_4 = json.load(f)

m_1_DD_result=DeepDiffMetric.evaluate_answers(P, G_1, df_modifying_parsed_1,ignore_keys=["homepage","collaborators","scripts"]) # update the path!!!!!更改路径tabular/config
m_1_DD_result_modifying=m_1_DD_result[['participantId','format','precision','recall','f1']]
m_1_DD_result_modifying['task']='modifying-task-tabular-1' # update the path!!!!!更改路径tabular/config

m_2_DD_result=DeepDiffMetric.evaluate_answers(P, G_2, df_modifying_parsed_2,ignore_keys=["homepage","collaborators","scripts"]) # update the path!!!!!更改路径tabular/config
m_2_DD_result_modifying=m_2_DD_result[['participantId','format','precision','recall','f1']]
m_2_DD_result_modifying['task']='modifying-task-tabular-2' # update the path!!!!!更改路径tabular/config

m_3_DD_result=DeepDiffMetric.evaluate_answers(P, G_3, df_modifying_parsed_3,ignore_keys=["homepage","collaborators","scripts"]) # update the path!!!!!更改路径tabular/config
m_3_DD_result_modifying=m_3_DD_result[['participantId','format','precision','recall','f1']]
m_3_DD_result_modifying['task']='modifying-task-tabular-3' # update the path!!!!!更改路径tabular/config

m_4_DD_result=DeepDiffMetric.evaluate_answers(P, G_4, df_modifying_parsed_4,ignore_keys=["homepage","collaborators","scripts"]) # update the path!!!!!更改路径tabular/config
m_4_DD_result_modifying=m_4_DD_result[['participantId','format','precision','recall','f1']]
m_4_DD_result_modifying['task']='modifying-task-tabular-4' # update the path!!!!!更改路径tabular/config

m_DD_result_modifying=pd.concat([m_1_DD_result_modifying,m_2_DD_result_modifying,m_3_DD_result_modifying,m_4_DD_result_modifying])

### merge the result
semantic_merged = m_DD_result_modifying.merge(
    m_ts_result_modifying[['participantId','format','task','tree_similarity']],
    on=['participantId','format','task'],
    how='left'
)

parse_semantic_merged = semantic_merged.merge(df_modifying_parse_result,on=['participantId','format','task'],how='left')

full_merged=parse_semantic_merged.merge(df_modifying_norm,on=['participantId','format','task'],how='left')

df_modifying_all = combine_modifying_results(full_merged, modifying_empty)

df_modifying_all.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/tabular_result/modifying_result.csv')   # update the path!!!!!更改路径tabular/config
df_modifying_all

/var/folders/z4/3vn4jgtj71n_hdpgm6b1r_ww0000gp/T/ipykernel_92058/1441401722.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_1_DD_result_modifying['task']='modifying-task-tabular-1' # update the path!!!!!更改路径tabular/config
/var/folders/z4/3vn4jgtj71n_hdpgm6b1r_ww0000gp/T/ipykernel_92058/1441401722.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_2_DD_result_modifying['task']='modifying-task-tabular-2' # update the path!!!!!更改路径tabular/config
/var/folders/z4/3vn4jgtj71n_hdpgm6b1r_ww0000gp/T/ipyke

,participantId,format,precision,recall,f1,task,tree_similarity,strict_parse,loose_parse,duration_sec,help_count,search_count,copy_count,paste_count,normalized_time
0,64457bc906c125cebd4bf66b,json5,1.000000,0.75,0.857143,modifying-task-tabular-1,0.994792,True,True,156.905,1,0,0,0,156.905
1,6697c8b6d9d7baa7edd8c4d3,hjson,1.000000,1.00,1.000000,modifying-task-tabular-1,1.000000,True,False,311.642,0,0,0,0,311.642
2,66475d6f7b27e4443ef0d031,json,0.000000,0.00,0.000000,modifying-task-tabular-1,0.979167,True,True,85.266,0,0,0,0,85.266
3,67f5bb69760b211cdfe8021c,jsonc,1.000000,1.00,1.000000,modifying-task-tabular-1,1.000000,True,True,182.869,0,0,0,0,182.869
4,67e571bef6af7ecd9c29ad72,json,0.666667,0.50,0.571429,modifying-task-tabular-1,0.979167,True,True,283.731,0,0,0,0,283.731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,6787d648d7b1963c661f9443,json5,0.000000,0.00,0.000000,modifying-task-tabular-2,0.000000,False,False,6.643,0,0,0,0,6.643
424,6787d648d7b1963c661f9443,json5,0.000000,0.00,0.000000,modifying-task-tabular-4,0.000000,False,False,18.339,0,0,0,0,18.339
425,66a662a0a709312db9de01f2,jsonc,0.000000,0.00,0.000000,modifying-task-tabular-1,0.000000,False,False,84.704,0,0,0,0,84.704
426,66a662a0a709312db9de01f2,jsonc,0.000000,0.00,0.000000,modifying-task-tabular-2,0.000000,False,False,22.852,0,0,0,0,22.852


In [28]:
full_merged_trees=full_merged[['participantId','format','task','tree_similarity','strict_parse','loose_parse','duration_sec','help_count','search_count','paste_count','normalized_time']]
full_merged_trees

,participantId,format,task,tree_similarity,strict_parse,loose_parse,duration_sec,help_count,search_count,paste_count,normalized_time
0,64457bc906c125cebd4bf66b,json5,modifying-task-tabular-1,0.994792,True,True,156.905,1,0,0,156.905
1,6697c8b6d9d7baa7edd8c4d3,hjson,modifying-task-tabular-1,1.000000,True,False,311.642,0,0,0,311.642
2,66475d6f7b27e4443ef0d031,json,modifying-task-tabular-1,0.979167,True,True,85.266,0,0,0,85.266
3,67f5bb69760b211cdfe8021c,jsonc,modifying-task-tabular-1,1.000000,True,True,182.869,0,0,0,182.869
4,67e571bef6af7ecd9c29ad72,json,modifying-task-tabular-1,0.979167,True,True,283.731,0,0,0,283.731
...,...,...,...,...,...,...,...,...,...,...,...
385,66bb6b7e0ccbed81f3ed30a8,xml,modifying-task-tabular-4,0.681592,True,True,127.223,0,0,0,127.223
386,6753746b1f8827e2bd586bf0,xml,modifying-task-tabular-4,0.716418,True,True,103.094,0,0,0,103.094
387,669a84e18da4349ece0ea67c,xml,modifying-task-tabular-4,0.855721,False,True,341.632,0,0,8,341.632
388,67ab57e88fd0774428bd1f32,xml,modifying-task-tabular-4,0.651741,False,True,566.347,0,0,0,566.347


In [31]:
import re
import json
from typing import Any, Dict, List
import pandas as pd

# 第三方库（请确保已 pip install）
import demjson3         # 最宽松 JSON
import hjson            # HJSON/JSONC
import json5            # JSON5
import yaml             # YAML
import toml             # TOML
import xmltodict        # XML → dict
from lxml import etree  # XML 恢复

# ─────────────── 1) 最宽容的格式解析 ───────────────

def parse_json_family(text: str) -> Dict[str,Any]:
    """依次用 demjson3, hjson, json5, 原生 json 尝试解析 JSON 系列格式。"""
    for loader in (
        lambda s: demjson3.decode(s),
        lambda s: hjson.loads(s),
        lambda s: json5.loads(s),
        lambda s: json.loads(s),
    ):
        try:
            obj = loader(text)
            if isinstance(obj, dict):
                return obj
            if isinstance(obj, list):
                return {"items": obj}
            return {"value": obj}
        except Exception:
            continue
    return {}

def parse_yaml(text: str) -> Dict[str,Any]:
    """YAML: library 优先，失败时行级 key: value 正则兜底。"""
    try:
        out = yaml.safe_load(text)
        return out if isinstance(out, dict) else {}
    except Exception:
        return {
            m.group(1): _coerce_primitive(m.group(2))
            for m in re.finditer(r"^(\w+)\s*:\s*(.+)$", text, re.MULTILINE)
        }

def parse_toml(text: str) -> Dict[str,Any]:
    """TOML: library 优先，失败时行级 key = value 正则兜底。"""
    try:
        return toml.loads(text)
    except Exception:
        return {
            m.group(1): _coerce_primitive(m.group(2))
            for m in re.finditer(r"^(\w+)\s*=\s*(.+)$", text, re.MULTILINE)
        }

def _etree_to_dict(node: etree._Element) -> Dict[str,Any]:
    """lxml 元素递归转 dict（attributes + children）。"""
    out: Dict[str,Any] = {}
    # 属性
    for k, v in node.attrib.items():
        out[f"@{k}"] = v
    # 子节点
    for child in node:
        tag = child.tag
        val = _etree_to_dict(child) if list(child) else (child.text or "")
        if tag in out:
            if isinstance(out[tag], list):
                out[tag].append(val)
            else:
                out[tag] = [out[tag], val]
        else:
            out[tag] = val
    return out

def parse_xml(text: str) -> Dict[str,Any]:
    """XML: xmltodict → lxml.recover → 行级正则兜底。"""
    cleaned = re.sub(r"<\?xml.*?\?>|<!--.*?-->", "", text, flags=re.DOTALL)
    # 1) xmltodict
    try:
        d = xmltodict.parse(cleaned, dict_constructor=dict)
        if isinstance(d, dict):
            # 删掉外层唯一键（通常是 root）
            return next(iter(d.values()))  # 取第一个 value
        return d
    except Exception:
        pass
    # 2) lxml recover
    try:
        parser = etree.XMLParser(recover=True)
        root = etree.fromstring(cleaned.encode(), parser)
        return _etree_to_dict(root)
    except Exception:
        pass
    # 3) 最简单回退：提取 <k>v</k>
    return {k: v for k, v in re.findall(r"<(\w+)>([^<]+)</\1>", text)}

def _coerce_primitive(val: str) -> Any:
    """辅助：把简单字面量转 int/float/bool/None，否则原样返回字符串。"""
    v = val.strip().strip('"').strip("'")
    if re.fullmatch(r"\d+", v):
        return int(v)
    if re.fullmatch(r"\d+\.\d+", v):
        return float(v)
    lv = v.lower()
    if lv in ("true","false"):
        return lv == "true"
    if lv in ("null","none"):
        return None
    return v

# ─────────────── 2) Dispatcher ───────────────

def parse_code(code: Any, fmt: str) -> Dict[str,Any]:
    """根据 fmt 调度到不同解析器。"""
    if not isinstance(code, str):
        return {}
    fmt = fmt.lower().strip()
    if fmt in ("json","jsonc","hjson","json5"):
        return parse_json_family(code)
    if fmt in ("yaml","yml"):
        return parse_yaml(code)
    if fmt == "toml":
        return parse_toml(code)
    if fmt == "xml":
        return parse_xml(code)
    return {}

# ─────────────── 3) 扁平化任意树结构 ───────────────

def flatten(obj: Any, path: str = "") -> Dict[str,Any]:
    """
    将 dict/list 递归展平为 path→leaf_value：
      - dict: 用 `.key` 
      - list: 用 `[i]`
    """
    out: Dict[str,Any] = {}
    if isinstance(obj, dict):
        for k, v in obj.items():
            p = f"{path}.{k}" if path else k
            out.update(flatten(v, p))
    elif isinstance(obj, list):
        for i, v in enumerate(obj):
            p = f"{path}[{i}]"
            out.update(flatten(v, p))
    else:
        out[path] = obj
    return out

# ─────────────── 4) 计算 PRF ───────────────

def compute_prf(
    P: Dict[str,Any],
    G: Dict[str,Any],
    A: Dict[str,Any]
) -> Dict[str,Any]:
    fP, fG, fA = flatten(P), flatten(G), flatten(A)
    setP, setG, setA = set(fP), set(fG), set(fA)

    # GT 操作
    gt_add = setG - setP
    gt_rm  = setP - setG
    gt_ch  = {k for k in (setP & setG) if fP[k] != fG[k]}

    # A 操作
    a_add = setA - setP
    a_rm  = setP - setA
    a_ch  = {k for k in (setP & setA) if fP[k] != fA[k]}

    D_GT = ({f"add:{k}"    for k in gt_add} |
            {f"remove:{k}" for k in gt_rm}  |
            {f"change:{k}" for k in gt_ch})
    D_A  = ({f"add:{k}"    for k in a_add} |
            {f"remove:{k}" for k in a_rm}  |
            {f"change:{k}" for k in a_ch})

    TP = D_A & D_GT
    FP = D_A - D_GT
    FN = D_GT - D_A

    prec = len(TP)/(len(TP)+len(FP)) if (TP or FP) else 0.0
    rec  = len(TP)/(len(TP)+len(FN)) if (TP or FN) else 0.0
    f1   = 2*prec*rec/(prec+rec)    if (prec+rec) else 0.0

    return {
        "precision": prec,
        "recall":    rec,
        "f1":        f1,
        "TP":        TP,
        "FP":        FP,
        "FN":        FN,
    }

# ─────────────── 5) 批量评估主函数 ───────────────
def evaluate_answers(
    P: Dict[str,Any],
    G: Dict[str,Any],
    user_df: pd.DataFrame,
    fmt_col: str = "format",
    ans_col: str = "answer",
    task_col: str = "task"
) -> pd.DataFrame:
    """
    对 user_df 中每行 answer（json/jsonc/hjson/json5/yaml/toml/xml）：
      1) parse_code → dict A
      2) compute_prf(P, G, A)
      3) 汇总 participantId, format, task, precision, recall, f1, TP, FP, FN
    """
    records: List[Dict[str,Any]] = []
    for _, row in user_df.iterrows():
        pid  = row["participantId"]
        fmt  = row[fmt_col]
        task = row[task_col]
        A    = parse_code(row[ans_col], fmt)
        m    = compute_prf(P, G, A)

        # 构造一条记录，task 也加入
        rec = {
            "participantId": pid,
            fmt_col:        fmt,
            task_col:       task,
            **m
        }
        records.append(rec)

    return pd.DataFrame(records)



df_modifying_1=modifying_nonempty[modifying_nonempty['task']=='modifying-task-tabular-1']
result_1= evaluate_answers(P, G_1, df_modifying_1)
result_1=result_1[['participantId','format','f1','task']]

df_modifying_2=modifying_nonempty[modifying_nonempty['task']=='modifying-task-tabular-2']
result_2= evaluate_answers(P, G_2, df_modifying_2)
result_2=result_2[['participantId','format','f1','task']]

df_modifying_3=modifying_nonempty[modifying_nonempty['task']=='modifying-task-tabular-3']
result_3= evaluate_answers(P, G_3, df_modifying_3)
result_3=result_3[['participantId','format','f1','task']]

df_modifying_4=modifying_nonempty[modifying_nonempty['task']=='modifying-task-tabular-4']
result_4= evaluate_answers(P, G_4, df_modifying_4)
result_4=result_4[['participantId','format','f1','task']]

result_f1=pd.concat([result_1,result_2,result_3,result_4])
result_f1

,participantId,format,f1,task
0,64457bc906c125cebd4bf66b,json5,0.857143,modifying-task-tabular-1
1,6697c8b6d9d7baa7edd8c4d3,hjson,1.000000,modifying-task-tabular-1
2,67e25388d573ec2d5f1c8442,yaml,1.000000,modifying-task-tabular-1
3,67e5cf90b79d0874e1cf16b0,toml,0.421053,modifying-task-tabular-1
4,67d00f24466a67ba7e41228e,toml,0.421053,modifying-task-tabular-1
...,...,...,...,...
94,67ab650c320eccf6141d4010,jsonc,0.115663,modifying-task-tabular-4
95,65fad673a546ab14baf2f8d3,json5,1.000000,modifying-task-tabular-4
96,67f1c7bd6f1c4a3d8deae02f,toml,0.073579,modifying-task-tabular-4
97,67adb27bd5f5776fcb16da62,hjson,0.653061,modifying-task-tabular-4


In [46]:
s=result_f1[result_f1['task']=='modifying-task-tabular-1']
s[s['format']=='xml']

,participantId,format,f1,task
10,669b961e1c9cf30a26bb57d2,xml,0.0,modifying-task-tabular-1
13,60721c2da01b3a616edba9a7,xml,0.0,modifying-task-tabular-1
26,67d95269b47464c18da766ca,xml,0.0,modifying-task-tabular-1
34,6740ae39809389d7da9de209,xml,0.0,modifying-task-tabular-1
37,62fd14d7b1e0729dc8aadb79,xml,0.0,modifying-task-tabular-1
52,677c52fe648cd68583dd3b1e,xml,0.0,modifying-task-tabular-1
60,67eff2b1a264260e948a4d14,xml,0.0,modifying-task-tabular-1
63,6501c0a2e788b5556c24569d,xml,0.0,modifying-task-tabular-1
68,66bb6b7e0ccbed81f3ed30a8,xml,0.0,modifying-task-tabular-1
69,6753746b1f8827e2bd586bf0,xml,0.0,modifying-task-tabular-1


In [53]:
import re
import json
from typing import Any, Dict, Tuple, List, Set
import pandas as pd

# 以下第三方库请提前安装
import demjson3
import hjson
import json5

import yaml
import toml

import xmltodict
from lxml import etree

# ─────────────── 1) 最宽容的格式解析 ───────────────

def parse_json_family(text: str) -> Dict[str,Any]:
    """对 JSONC/HJSON/JSON5/原生 JSON 依次尝试最宽容的解析器。"""
    for loader in (
        lambda s: demjson3.decode(s),
        lambda s: hjson.loads(s),
        lambda s: json5.loads(s),
        lambda s: json.loads(s),
    ):
        try:
            obj = loader(text)
            # 只要是 dict 或 list 我们都包成 dict
            if isinstance(obj, dict):
                return obj
            if isinstance(obj, list):
                return {"items": obj}
            return {"value": obj}
        except Exception:
            continue
    return {}

def parse_yaml(text: str) -> Dict[str,Any]:
    """YAML: 库 + 回退正则"""
    try:
        out = yaml.safe_load(text)
        return out if isinstance(out, dict) else {}
    except Exception:
        # 简单行级 key:value
        return {m.group(1): _coerce_primitive(m.group(2))
                for m in re.finditer(r"^(\w+)\s*:\s*(.+)$", text, re.MULTILINE)}

def parse_toml(text: str) -> Dict[str,Any]:
    """TOML: 库 + 回退正则"""
    try:
        return toml.loads(text)
    except Exception:
        return {m.group(1): _coerce_primitive(m.group(2))
                for m in re.finditer(r"^(\w+)\s*=\s*(.+)$", text, re.MULTILINE)}

def _coerce_xml_types(x: Any) -> Any:
    """
    递归把字符串数字/布尔/空/null/etc 转回对应 Python 原生类型
    并删掉空字符串
    """
    if isinstance(x, dict):
        out = {}
        for k, v in x.items():
            v2 = _coerce_xml_types(v)
            # 如果子节点全部变成 None，就跳过
            if v2 is None:
                continue
            out[k] = v2
        return out or None
    if isinstance(x, list):
        lst = [_coerce_xml_types(v) for v in x]
        # 过滤掉 None
        lst = [v for v in lst if v is not None]
        return lst or None
    # leaf node: text
    if isinstance(x, str):
        s = x.strip()
        if not s:
            return None
        # 整数
        if re.fullmatch(r"\d+", s):
            return int(s)
        # 浮点
        if re.fullmatch(r"\d+\.\d+", s):
            return float(s)
        # 布尔
        ls = s.lower()
        if ls in ("true","false"):
            return ls == "true"
        # null/none
        if ls in ("null","none"):
            return None
        return s
    # 其他类型不变
    return x

def parse_xml(text: str) -> Dict[str,Any]:
    """
    最先用 xmltodict，强制 movie 标签总是列表；
    失败时尝试 lxml 的 recover，再失败时 regex 简单降级。
    最后统一做类型强转。
    """
    # 1) 去掉 prolog/注释
    cleaned = re.sub(r"<\?xml.*?\?>|<!DOCTYPE.*?>|<!--.*?-->", "", text, flags=re.DOTALL)

    # 2) xmltodict with force_list
    try:
        d = xmltodict.parse(
            cleaned,
            dict_constructor=dict,
            force_list=("movie",)    # 如果你的根里还有别的重复标签，也加进来
        )
        # 如果顶层只有一个键，就取它
        root = next(iter(d.values())) if isinstance(d, dict) else d
        return _coerce_xml_types(root) or {}
    except Exception:
        pass

    # 3) lxml recover
    try:
        parser = etree.XMLParser(recover=True, remove_comments=True)
        root = etree.fromstring(cleaned.encode("utf-8"), parser=parser)
        # 转 dict（跟你之前的 _etree_to_dict 类似）
        def _etree_to_dict(node):
            obj = {}
            # attributes
            for k,v in node.attrib.items():
                obj[f"@{k}"] = v
            # children
            for child in node:
                tag = child.tag
                val = _etree_to_dict(child) if list(child) else (child.text or "")
                existing = obj.get(tag)
                if existing is None:
                    obj[tag] = val
                elif isinstance(existing, list):
                    existing.append(val)
                else:
                    obj[tag] = [existing, val]
            return obj

        parsed = _etree_to_dict(root)
        return _coerce_xml_types(parsed) or {}
    except Exception:
        pass

    # 4) 最后降级正则：只提取最简单的 <tag>value</tag>
    fallback = {}
    for tag, val in re.findall(r"<(\w+)>\s*([^<]+?)\s*</\1>", text):
        v = _coerce_xml_types(val)
        if v is not None:
            if tag in fallback:
                if isinstance(fallback[tag], list):
                    fallback[tag].append(v)
                else:
                    fallback[tag] = [fallback[tag], v]
            else:
                fallback[tag] = v
    return fallback

def _etree_to_dict(node: etree._Element) -> Dict[str,Any]:
    out: Dict[str,Any] = {}
    # attributes
    for k,v in node.attrib.items():
        out[f"@{k}"] = v
    # children
    for child in node:
        tag = child.tag
        val = _etree_to_dict(child) if list(child) else (child.text or "")
        if tag in out:
            if isinstance(out[tag], list):
                out[tag].append(val)
            else:
                out[tag] = [out[tag], val]
        else:
            out[tag] = val
    return out

def _coerce_primitive(val: str) -> Any:
    """辅助: 把简单字面量转 int/bool/None"""
    v = val.strip().strip('"').strip("'")
    if re.fullmatch(r"\d+", v):
        return int(v)
    if re.fullmatch(r"\d+\.\d+", v):
        return float(v)
    lv = v.lower()
    if lv in ("true","false"):
        return lv=="true"
    if lv in ("null","none"):
        return None
    return v

# ─────────────── 2) Dispatcher: 根据 format 选解析器 ───────────────

def parse_code(code: Any, fmt: str) -> Dict[str,Any]:
    if not isinstance(code, str):
        return {}
    fmt = fmt.lower().strip()
    if fmt in ("json","jsonc","hjson","json5"):
        return parse_json_family(code)
    if fmt in ("yaml","yml"):
        return parse_yaml(code)
    if fmt == "toml":
        return parse_toml(code)
    if fmt == "xml":
        return parse_xml(code)
    # fallback 空
    return {}

# ─────────────── 3) 扁平化任意树结构 ───────────────

def flatten(obj: Any, path: str="") -> Dict[str,Any]:
    """
    把 dict/list 递归摊平成 path→leaf_value。
    list 用 [i] 标识，dict 用 .key 标识。
    """
    out: Dict[str,Any] = {}
    if isinstance(obj, dict):
        for k,v in obj.items():
            p = f"{path}.{k}" if path else k
            out.update(flatten(v, p))
    elif isinstance(obj, list):
        for i, v in enumerate(obj):
            p = f"{path}[{i}]"
            out.update(flatten(v, p))
    else:
        out[path] = obj
    return out

# ─────────────── 4) 计算 PRF ───────────────

def compute_prf(P: Dict[str,Any], G: Dict[str,Any], A: Dict[str,Any]) -> Dict[str,Any]:
    fP, fG, fA = flatten(P), flatten(G), flatten(A)

    setP, setG, setA = set(fP), set(fG), set(fA)
    # GT ops
    gt_add = setG - setP
    gt_rm  = setP - setG
    gt_ch  = {k for k in setP & setG if fP[k] != fG[k]}
    # A ops
    a_add = setA - setP
    a_rm  = setP - setA
    a_ch  = {k for k in setP & setA if fP[k] != fA[k]}

    D_GT = {f"add:{k}" for k in gt_add} | {f"remove:{k}" for k in gt_rm} | {f"change:{k}" for k in gt_ch}
    D_A  = {f"add:{k}" for k in a_add} | {f"remove:{k}" for k in a_rm} | {f"change:{k}" for k in a_ch}

    TP = D_A & D_GT
    FP = D_A - D_GT
    FN = D_GT - D_A

    prec = len(TP)/(len(TP)+len(FP)) if TP or FP else 0.0
    rec  = len(TP)/(len(TP)+len(FN)) if TP or FN else 0.0
    f1   = 2*prec*rec/(prec+rec) if (prec+rec) else 0.0

    return {"precision":prec, "recall":rec, "f1":f1, "TP":TP, "FP":FP, "FN":FN}

# ─────────────── 5) 批量评估主函数 ───────────────

def evaluate_answers(
    P: Dict[str,Any],
    G: Dict[str,Any],
    user_df: pd.DataFrame,
    fmt_col: str="format",
    ans_col: str="answer"
) -> pd.DataFrame:
    """
    对 user_df 中每行 answer（xml/yaml/toml/json 系列）:
      1) parse_code → dict
      2) compute_prf(P, G, A)
      3) 汇总返回 DataFrame
    """
    rows: List[Dict[str,Any]] = []
    for _, r in user_df.iterrows():
        pid = r["participantId"]
        fmt = r[fmt_col]
        A   = parse_code(r[ans_col], fmt)
        m   = compute_prf(P, G, A)
        rows.append({"participantId":pid, fmt_col:fmt, **m})
    return pd.DataFrame(rows)

# ─────────────── 6) 使用示例 ───────────────


    # 3) 筛出需要的格式
df_modifying_1=modifying_nonempty[modifying_nonempty['task']=='modifying-task-tabular-1']
sub_1 = df_modifying_1[df_modifying_1["format"].isin(["xml","yaml","toml"])]

result_1= evaluate_answers(P, G_1, sub_1)
result_1=result_1[['participantId','format','f1']]

df_modifying_2=modifying_nonempty[modifying_nonempty['task']=='modifying-task-tabular-2']
sub_2 = df_modifying_2[df_modifying_2["format"].isin(["xml","yaml","toml"])]

result_2= evaluate_answers(P, G_2, sub_2)
result_2=result_2[['participantId','format','f1']]

df_modifying_3=modifying_nonempty[modifying_nonempty['task']=='modifying-task-tabular-3']
sub_3 = df_modifying_3[df_modifying_3["format"].isin(["xml","yaml","toml"])]

result_3= evaluate_answers(P, G_3, sub_3)
result_3=result_3[['participantId','format','f1']]

df_modifying_4=modifying_nonempty[modifying_nonempty['task']=='modifying-task-tabular-4']
sub_4 = df_modifying_4[df_modifying_4["format"].isin(["xml","yaml","toml"])]

result_4= evaluate_answers(P, G_4, sub_4)
result_4=result_3[['participantId','format','f1']]

result_xty=pd.concat([result_1,result_2,result_3,result_4])
result_xty


,participantId,format,f1
0,67e25388d573ec2d5f1c8442,yaml,1.000000
1,67e5cf90b79d0874e1cf16b0,toml,0.421053
2,67d00f24466a67ba7e41228e,toml,0.421053
3,669b961e1c9cf30a26bb57d2,xml,0.000000
4,66992f58a7964f557fc75f58,toml,0.421053
...,...,...,...
38,66a9285501ac98053902d481,toml,0.881720
39,5d766978f47e6200019a2acf,yaml,0.677419
40,677d32c679491be18090225b,xml,0.277966
41,665c5d94303fdf202ac9ded0,toml,0.881720


In [56]:
result_1[result_1['format']=='xml']

,participantId,format,f1
3,669b961e1c9cf30a26bb57d2,xml,0.0
5,60721c2da01b3a616edba9a7,xml,0.0
8,67d95269b47464c18da766ca,xml,0.0
11,6740ae39809389d7da9de209,xml,0.0
13,62fd14d7b1e0729dc8aadb79,xml,0.0
20,677c52fe648cd68583dd3b1e,xml,0.0
22,67eff2b1a264260e948a4d14,xml,0.0
23,6501c0a2e788b5556c24569d,xml,0.0
26,66bb6b7e0ccbed81f3ed30a8,xml,0.0
27,6753746b1f8827e2bd586bf0,xml,0.0


In [8]:
m_1_DD_result[m_1_DD_result['format']=='yaml']

,participantId,format,precision,recall,f1,TP,FP,FN
59,67e25388d573ec2d5f1c8442,yaml,0.0,0.0,0.0,{},{},"{change:root['movies']['movie'][3]['writer'], ..."
63,67dc19c31078d1581e39e75a,yaml,0.0,0.0,0.0,{},{},"{change:root['movies']['movie'][3]['writer'], ..."
64,67adb520391eb17f214a50b4,yaml,0.0,0.0,0.0,{},{},"{change:root['movies']['movie'][3]['writer'], ..."
66,66049694dfb8828a8f3dce70,yaml,0.0,0.0,0.0,{},{},"{change:root['movies']['movie'][3]['writer'], ..."
68,6787e4bd2bf9557a90208f03,yaml,0.0,0.0,0.0,{},{},"{change:root['movies']['movie'][3]['writer'], ..."
70,67f184a3b82990251abd1592,yaml,0.0,0.0,0.0,{},{},"{change:root['movies']['movie'][3]['writer'], ..."
72,60fce3df91350150e2e88423,yaml,0.0,0.0,0.0,{},{},"{change:root['movies']['movie'][3]['writer'], ..."
73,67d29234dbfdb6668d124115,yaml,0.0,0.0,0.0,{},{},"{change:root['movies']['movie'][3]['writer'], ..."
74,671a519900a99ebc15a67672,yaml,0.0,0.0,0.0,{},{},"{change:root['movies']['movie'][3]['writer'], ..."
75,637d8ca00150ac6e71a34ec9,yaml,0.0,0.0,0.0,{},{},"{change:root['movies']['movie'][3]['writer'], ..."


In [ ]:
df_modifying_parsed_1[df_modifying_parsed_1['format']=='toml']

In [6]:
import pprint

pd.set_option('display.max_colwidth', None)

# 2. 找到你要调试的那个 participantId：
pid = '67e571bef6af7ecd9c29ad72'  # 换成你实际想看的 id

# 3. 取出那一行：
row = df_modifying_parsed_1.loc[df_modifying_parsed_1['participantId'] == pid]
parsed = row.iloc[0]['parsed_answer']
print(f"---- parsed_answer for participantId={pid!r} ----\n")
pprint.pprint(parsed, width=200)
print("\n----------------------------------------------")

---- parsed_answer for participantId='67e571bef6af7ecd9c29ad72' ----

{'movies': {'movie': [{'awards': [],
                       'box_office': None,
                       'budget': 500,
                       'cast': [],
                       'cinematography': 'Unknown',
                       'country': 'United States',
                       'director': 'James H. White',
                       'genres': ['Silent', 'Short'],
                       'href': None,
                       'language': 'Silent',
                       'production_company': 'Edison Manufacturing Company',
                       'release_date': '1900-03-15',
                       'runtime': 3,
                       'title': 'After Dark in Central Park',
                       'writer': 'James H. White',
                       'year': 1900},
                      {'awards': [],
                       'box_office': None,
                       'budget': 450,
                       'cast': [],
              

In [13]:
DeepDiffMetric.debug_participant(
    participant_id="60721c2da01b3a616edba9a7",
    P=P,
    G=G_1,
    answer_df=df_modifying_parsed_1,
    parsed_col="parsed_answer",      # 如果你的列名叫 parsed_code
    id_col="participantId",
    fmt_col="format",            # 若需要忽略某些 key
)


DEBUG participantId = 60721c2da01b3a616edba9a7  format = xml

1) canonicalized P:
 {
  "movies": {
    "movie": [
      {
        "title": "After Dark in Central Park",
        "year": 1900,
        "cast": [],
        "genres": [
          "Silent",
          "Short"
        ],
        "href": null,
        "country": "United States",
        "runtime": 3,
        "language": "Silent",
        "release_date": "1900-03-15",
        "production_company": "Edison Manufacturing Company",
        "director": "James H. White",
        "writer": "Unknown",
        "cinematography": "Unknown",
        "budget": 500,
        "box_office": null,
        "awards": []
      },
      {
        "title": "Boarding School Girls' Pajama Parade",
        "year": 1900,
        "cast": [],
        "genres": [
          "Silent",
          "Short"
        ],
        "href": null,
        "country": "United States",
        "runtime": 2,
        "language": "Silent",
        "release_date": "1900-05-10",


In [ ]:
df_1=df_modifying_all[df_modifying_all['task']=='modifying-task-tabular-3']
df_1[df_1['format']=='hjson']['f1']

In [12]:
df_modifying_parsed_1

,participantId,format,task,parsed_answer
0,64457bc906c125cebd4bf66b,json5,modifying-task-tabular-1,{'movies': {'movie': [{'title': 'After Dark in...
4,6697c8b6d9d7baa7edd8c4d3,hjson,modifying-task-tabular-1,"{'movies': {'movie': [OrderedDict([('title', '..."
7,66475d6f7b27e4443ef0d031,json,modifying-task-tabular-1,{'movies': {'movie': [{'title': 'After Dark in...
11,67f5bb69760b211cdfe8021c,jsonc,modifying-task-tabular-1,{'movies': {'movie': [{'title': 'After Dark in...
15,67e571bef6af7ecd9c29ad72,json,modifying-task-tabular-1,{'movies': {'movie': [{'title': 'After Dark in...
...,...,...,...,...
368,6501c0a2e788b5556c24569d,xml,modifying-task-tabular-1,{'movies': {'movie': [{'title': 'After Dark in...
372,66bb6b7e0ccbed81f3ed30a8,xml,modifying-task-tabular-1,{'movies': {'movie': [{'title': 'After Dark in...
376,6753746b1f8827e2bd586bf0,xml,modifying-task-tabular-1,{'movies': {'movie': [{'title': 'After Dark in...
380,669a84e18da4349ece0ea67c,xml,modifying-task-tabular-1,{'movies': {'movie': [{'title': 'After Dark in...


In [13]:
import json
row = df_modifying_parsed_1.iloc[98]
print(json.dumps(row['parsed_answer'], ensure_ascii=False, indent=2))

{
  "movies": {
    "movie": [
      {
        "title": "After Dark in Central Park",
        "year": "1900",
        "cast": null,
        "genres": [
          "Silent",
          "Short"
        ],
        "href": null,
        "country": "United States",
        "runtime": "3",
        "language": "Silent",
        "release_date": "1900-03-15",
        "production_company": "Edison Manufacturing Company",
        "director": "James H. White",
        "writer": "James H. White",
        "cinematography": "Unknown",
        "budget": "500",
        "box_office": null,
        "awards": null
      },
      {
        "title": "Boarding School Girls' Pajama Parade",
        "year": "1900",
        "cast": null,
        "genres": [
          "Silent",
          "Short"
        ],
        "href": null,
        "country": "United States",
        "runtime": "2",
        "language": "Silent",
        "release_date": "1900-05-10",
        "production_company": "Edison Manufacturing Company",

### quiz

In [ ]:
quiz_results_result=quiz_results[['participantId','format','quiz_key','correct','num_wrong_attempts','duration_sec','wrong_choice_count']]
quiz_results_result.rename(columns={'quiz_key':'task'},inplace=True)
quiz_results_result.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/config_result/quiz_result.csv')
quiz_results_result

In [3]:
quiz_results_result=quiz_results[['participantId','format','quiz_key','correct','num_wrong_attempts','duration_sec','wrong_choice_count']]
quiz_results_result.rename(columns={'quiz_key':'task'},inplace=True)
quiz_results_result.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/tabular_result/quiz_result.csv')
quiz_results_result

/var/folders/z4/3vn4jgtj71n_hdpgm6b1r_ww0000gp/T/ipykernel_92058/1265095582.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quiz_results_result.rename(columns={'quiz_key':'task'},inplace=True)


,participantId,format,task,correct,num_wrong_attempts,duration_sec,wrong_choice_count
0,64457bc906c125cebd4bf66b,json5,tutorial-part1,1,3,225.255,4
1,64457bc906c125cebd4bf66b,json5,tutorial-part2,0,5,128.872,0
2,6697c8b6d9d7baa7edd8c4d3,hjson,tutorial-part1,1,3,370.784,3
3,6697c8b6d9d7baa7edd8c4d3,hjson,tutorial-part2,1,1,100.488,0
4,67e25388d573ec2d5f1c8442,yaml,tutorial-part1,1,0,230.849,0
...,...,...,...,...,...,...,...
209,67f1c7bd6f1c4a3d8deae02f,toml,tutorial-part2,1,0,133.142,0
210,67adb27bd5f5776fcb16da62,hjson,tutorial-part1,1,3,662.848,5
211,67adb27bd5f5776fcb16da62,hjson,tutorial-part2,1,1,143.695,0
212,6675c40cdc52b37294f0514e,hjson,tutorial-part1,0,5,1096.291,4


### nasa

In [5]:
df_nasa_clean=df_nasa[['participantId','format','duration_sec','mental-demand','physical-demand','temporal-demand','performance','frustration','effort']]
df_nasa_clean.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/config_result/nasa_result.csv')

In [4]:
df_nasa_clean=df_nasa[['participantId','format','duration_sec','mental-demand','physical-demand','temporal-demand','performance','frustration','effort']]
df_nasa_clean.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/tabular_result/nasa_result.csv')

### post_survey

In [7]:
df_post.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/config_result/post_result.csv')

In [5]:
df_post.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/tabular_result/post_result.csv')

### con + diff

In [ ]:
df_dif_conf_result=df_dif_conf[['participantId','format','duration_sec','task','confidence','difficulty']]
df_dif_conf_result.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/config_result/dif_conf_result.csv')
df_dif_conf_result

In [6]:
df_dif_conf_result=df_dif_conf[['participantId','format','duration_sec','task','confidence','difficulty']]
df_dif_conf_result.to_csv('/Users/shiyi.he/Desktop/DSF_Analysis/tabular_result/dif_conf_result.csv')
df_dif_conf_result

,participantId,format,duration_sec,task,confidence,difficulty
0,64457bc906c125cebd4bf66b,json5,8.808,writing-task-NL,7,1
1,64457bc906c125cebd4bf66b,json5,5.996,reading-task-tabular-1,4,4
2,64457bc906c125cebd4bf66b,json5,10.567,reading-task-tabular-2,3,4
3,64457bc906c125cebd4bf66b,json5,7.485,reading-task-tabular-3,3,5
4,64457bc906c125cebd4bf66b,json5,5.386,reading-task-tabular-4,5,3
...,...,...,...,...,...,...
1172,6675c40cdc52b37294f0514e,hjson,9.406,writing-task-tabular,4,5
1173,6675c40cdc52b37294f0514e,hjson,7.851,modifying-task-tabular-1,6,2
1174,6675c40cdc52b37294f0514e,hjson,4.943,modifying-task-tabular-2,6,2
1175,6675c40cdc52b37294f0514e,hjson,6.186,modifying-task-tabular-3,1,7


In [ ]:
fig, sig_df = compare_tasks_across_formats(
    df=reading_aggregated_result,
    task_list=["reading-task-config-1", "reading-task-config-3"],
    compare_value="correct",   # or "duration_sec"
    data_type="categorical",   # "continuous" or "categorical"
    test_type="auto",          # 推荐保持 auto
    correction_method="holm"
)

In [ ]:
import json
row = df_all_0.iloc[1]
print(json.dumps(row['parsed_answer'], ensure_ascii=False, indent=2))